In [2]:
import gzip
import json
from collections import defaultdict

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
f = gzip.open('/content/drive/MyDrive/cse256_final_project_huayueli_chengxiguo/renttherunway_final_data.json.gz')
dataset = []
for l in f:
    dataset.append(json.loads(l))

There are 2565349 product reviews for "Video Games" in total.Therefore, we think this dataset is large enough.

In [5]:
len(dataset)

192544

In [6]:
dataset[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [8]:
allRatings = []
for i in dataset:
  if (i['rating'] is not None and i['user_id'] is not None and i['item_id'] is not None):
    allRatings.append([i['user_id'],i['item_id'],float(i['rating'])])
allRatings[1]

['273551', '153475', 10.0]

In [9]:
#we set a part of the dataset as the vaild dataset to assess the validity of your model’s predictions (avoid overfitting and test the accuracy in the same time)
ratingsTrain = allRatings[:135000]
ratingsValid = allRatings[135000:173000]
ratingsTest = allRatings[173000:]

In [10]:
#for the baseline
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

all_kinds_ratings = []
userRatings = defaultdict(list)
'''
for i in allRatings:
    user = i[0]
    item = i[1]
    rating = i[2]
    all_kinds_ratings.append(rating)
    userRatings[user].append(rating)
'''

for i in ratingsTrain:
    user = i[0]
    item = i[1]
    rating = i[2]
    all_kinds_ratings.append(rating)
    userRatings[user].append(rating)

globalAverage = sum(all_kinds_ratings) / len(all_kinds_ratings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

In [11]:
ratingsTrain = allRatings[:135000]
ratingsValid = allRatings[135000:173000]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [12]:
ratingMean=0
for u,b,r in ratingsTrain:
    ratingMean+=r
ratingMean/=len(ratingsTrain)
ratingMean

9.093466666666666

In [13]:
alpha = ratingMean

In [14]:
N = len(ratingsTrain)
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())

In [15]:
#all funcs for model
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,b,r in ratingsTrain:

        pred = prediction(u, b)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[b] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[b] += 2*lamb*itemBiases[b]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[b] for b in items]
    return numpy.array(dtheta)

def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, b) for u,b,_ in ratingsTrain]
    cost = MSE(predictions, labels)
    # print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

def prediction(user, item):
    if user in userBiases and item in itemBiases:
        return alpha + userBiases[user] + itemBiases[item]
    elif item in itemBiases:
        return alpha + itemBiases[item]
    elif user in userBiases:
        return alpha + userBiases[user]
    else:
        return alpha

In [16]:
train_labels = [r for _,_,r in ratingsTrain]
valid_labels = [r for _,_,r in ratingsValid]

In [17]:
import scipy
import numpy

from sklearn.metrics import mean_squared_error

MSE=mean_squared_error

scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),derivative, args = (train_labels, 0.001))
predictions_sim=[]

for u,b,r in ratingsValid:
    predict=prediction(u, b)
    predictions_sim.append(predict)

train_loss = MSE(predictions_sim,valid_labels)
print("Train loss:", train_loss)

Train loss: 1.955558180053929


In [18]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),derivative, args = (train_labels, 0.001))
predictions_sim2=[]

for u,b,r in ratingsValid:
    predict=prediction(u, b)
    predictions_sim2.append(predict)

val_loss = MSE(predictions_sim2,valid_labels)
print("Validation loss:", val_loss)

Validation loss: 1.9594909779919119
